# IDS 561 - Homework 2 
# By Hoang Nha Nguyen 
# UIN: 671491808

Setting up Pyspark

In [10]:
!pip install -q findspark
!pip install pyspark

import findspark
findspark.init()

# Task 1

In [6]:
#Load the Amazon data
spark = SparkSession.builder.master("local[*]").getOrCreate()
data = spark.read.csv('Amazon_Responded_Oct05.csv',header = True)

In [2]:
from pyspark.sql import SparkSession                                                          # Creating a Spark Session
#Create Spark session                                                        
spark = SparkSession.builder.appName('xyz').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 14:17:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Step 1 : Remove the records where “user_verified” is “FALSE”.

In [12]:
# Loading the Amazon dataset

amazon = spark.read.csv("Amazon_Responded_Oct05.csv", header = "true",inferSchema=True,encoding='UTF-8',multiLine=True,escape="\"").select('id_str','tweet_created_at','user_verified','favorite_count','retweet_count','text_')


In [13]:
amazon1 = amazon.select('id_str','tweet_created_at','user_verified','favorite_count','retweet_count','text_')

amazon2 = amazon1.filter(amazon1.user_verified=='True')

amazon2Rdd = amazon2.rdd

#Convert csv to rdd
amazon2Rdd.count()

171899

# Step 2 : For the remaining records (“user_verified” is “TRUE”), group by created date, and count the number of tweets for each date.

In [15]:
#group by date
from datetime import datetime                                                                    

import pytz

def changeDate(x):

  date = x.split(' ')
  return date[1]+' '+ date[2]



In [16]:
ChangeDate = amazon2Rdd.map(lambda x: (x[0], changeDate(x[1]),x[2],x[3],x[4],x[5]))

ChangeDate.take(3)



[("'793281386912354304'",
  'Nov 01',
  True,
  0,
  0.0,
  "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL"),
 ("'793502854459879424'",
  'Nov 01',
  True,
  0,
  0.0,
  '@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB'),
 ("'793504235400884224'",
  'Nov 01',
  True,
  0,
  0.0,
  '@SeanEPanjab Without providing any sensitive information, do the order details indicate that the item was ever shipped? ^SW')]

In [17]:
#number of tweets for each date
groupRdd = ChangeDate.map(lambda x: (x[1],1)).groupByKey()                            

freq = groupRdd.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)

freq.toDF().show(30)

+----+------+
|  _1|    _2|
+----+------+
|1536|Jan 03|
|1508|Jan 10|
|1496|Jan 11|
|1410|Jan 12|
|1364|Jan 06|
|1360|Jan 07|
|1342|Jan 20|
|1298|Mar 02|
|1295|Jan 13|
|1292|Jan 21|
|1290|Jan 14|
|1286|Jan 18|
|1279|Dec 15|
|1259|Jan 24|
|1249|Nov 18|
|1201|Dec 03|
|1196|Jan 02|
|1192|Jun 27|
|1190|Jul 04|
|1175|Jan 19|
|1163|Jan 25|
|1149|Jan 23|
|1143|Jan 08|
|1124|Jan 17|
|1120|May 11|
|1112|Jul 03|
|1109|Mar 30|
|1089|Apr 05|
|1083|Jan 26|
|1080|Jan 27|
+----+------+
only showing top 30 rows



# Step 3 : For the date with highest number of tweets, calculate the sum of “favorite_count” and “retweet_count” for each tweet on that day. 

In [18]:
datafreq = freq.toDF()

maxcount = datafreq.agg({"_1": "max"}).rdd.collect()[0][0]

maxdate = datafreq.filter(datafreq._1== maxcount).select("_2").rdd.collect()[0][0]

print(maxdate)




Jan 03


In [21]:
# Then report the text content of the top 100 tweets with highest sum
import string                                                                                                  

data_ChangeDate = ChangeDate.toDF()

data_sum = data_ChangeDate.filter(data_ChangeDate._2==maxdate).withColumn("sum_tweet", data_ChangeDate._4+data_ChangeDate._5).sort('sum_tweet', ascending=False)

data_frequencyCount = data_sum.limit(100).rdd.flatMap(lambda x: x._6.split(" "))


In [26]:
# eliminate the integers, punctuations and coverting all words to lower case
def cleandata(x):                                                                                           
  
  lowercase = x.lower()
  punc = list(string.punctuation)
  
  for x in range(0, 9):
    punc.append(str(x))
  for i in punc:
    lowercase = lowercase.replace(i,'')
  return lowercase

cleandata = data_frequencyCount.map(cleandata)

# Eliminating the spaces
cleandata = cleandata.filter(lambda x:x!='')                                                                  




# using MapReduce to count word freq of 100 tweets

In [27]:
data_MapReduce = cleandata.map(lambda word: (word, 1)).reduceByKey(lambda v1, v2: v1+v2).sortByKey()          

for i in data_MapReduce.collect(): 
  print (i)

data_MapReduce1 = data_MapReduce.toDF().withColumnRenamed("_1","Word").withColumnRenamed("_2","Frequency")



('a', 24)
('ab', 1)
('abhinavsaroj', 4)
('able', 3)
('about', 4)
('aboutasix', 1)
('above', 2)
('access', 3)
('account', 2)
('accttypically', 1)
('add', 1)
('address', 1)
('adibaddy', 1)
('advise', 1)
('ae', 3)
('af', 2)
('aim', 2)
('ak', 1)
('alerts', 2)
('alexa', 1)
('alexisgraham99', 1)
('allow', 1)
('already', 1)
('always', 3)
('alyssagoldman', 3)
('am', 1)
('amazon', 7)
('amazonhelp', 1)
('amazonin', 2)
('ambermullennn', 1)
('amp', 1)
('an', 7)
('and', 17)
('any', 8)
('anything', 1)
('apologies', 2)
('app', 6)
('application', 1)
('appreciation', 1)
('ar', 3)
('are', 2)
('arent', 1)
('ariannadelbene', 1)
('arrange', 1)
('arrive', 6)
('arrived', 3)
('arrives', 1)
('as', 4)
('ask', 1)
('asked', 1)
('assist', 2)
('assistance', 1)
('at', 6)
('athena', 1)
('athenarivera', 1)
('atoz', 1)
('attention', 2)
('availability', 1)
('available', 2)
('aw', 1)
('awww', 1)
('babajabalpuri', 4)
('back', 1)
('bad', 2)
('baileyoz', 1)
('bayou', 1)
('bbyuniversecb', 1)
('be', 8)
('becoming', 1)
('been'

In [25]:
# Exporting to csv file 

data_MapReduce1.toPandas().to_csv('/Users/hoangnha218/Desktop/IDS561_HW2/Task1.csv',mode = 'w', index=False)

# Task 2

In [28]:
# Load find_text data file 
text = spark.read.csv("find_text.csv", header = "true")

In [29]:
# Merging Amazon data and text data on "id_str" to fill in the "text" column

amazon3 = amazon1.select('id_str', 'text_')

Amazon_Responded_Oct05 = amazon3.rdd

find_text = text.rdd

mergedRdd = find_text.join(Amazon_Responded_Oct05)

mergedRdd.collect()


finalRdd = mergedRdd.flatMap(lambda x: [(x[0], v) for v in x[1]])

df= finalRdd.toDF().withColumnRenamed("_1","id_str").withColumnRenamed("_2","text")

df1 = df.filter(df.text. isNotNull())


In [31]:
# Exporting to csv file

df1.toPandas().to_csv('/Users/hoangnha218/Desktop/IDS561_HW2/Task2.csv',mode = 'w', index=False)